In [47]:
# APRS-NWS
# Copyright 2021, Kurt Kochendarfer (KE7KUS)

# ===GNU Public License v3===
# This file is part of APRS-NWS.

# APRS-NWS is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License 
# as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

# APRS-NWS is distributed in the hope that it will be useful,but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

# You should have received a copy of the GNU General Public License along with APRS-NWS.  If not, see <https://www.gnu.org/licenses/>.

## License

APRS-NWS is distributed under the GNU Public License v3.  See the <a href="https://www.gnu.org/licenses">Free Software Foundation</a> for details.

In [48]:
#-----IMPORTS-----#
import time, datetime, requests, os, sys
import lxml.etree as et
from io import BytesIO
import nm, cap

## Program Dependencies

The Python libraries used to support this program and their functional overview are listed below:

* time - used to execute the *sleep* function between message build attempts.
* datetime - used in formatting the DTG for alert message start and end.
* requests - used in retrieving XML data from the NWS alerts website
* os - used to write the formatted text packet text file ('/tmp/wxalerts.txt')
* sys - used to write output to *stdout*
* lxml.etree - handles the XPath processing of the NWS ATOM feed
* io - used in byte type conversion for XML string handling

* nm - companion file from APRS-NWS package which has dictionary of NM zones/counties.  Used for NWS -> APRS formatting conversions.
* cap - companion file from APRS-NWS package which has several dictionaries of CAP-standard messages/events.  Used for NWS -> APRS formatting conversions.

Dependency resolution/installation is built into the software and should self-resolve.

In [49]:
#-----GLOBALS-----#
LOCALE = ('nm')
t = None

## Global Definitions

Several global definitions are needed:

* LOCALE - used in conjunction with import of .py file definitions for zones/counties to be processed by APRS-NWS
* t - declaration ultimately used to hold XML feed data

In [50]:
#-----CLASSES-----#
class XMLHandler:
    """Methods for extracting alert data from the NWS ATOM feed."""

    def __init__(self):
        """Initializes XML handler to parse imported XML (normally from an instance of the webHandler class). 
        
        Namespaces are unique to the XML file being parsed and are defined in the self.ns variable.  Even if only using a default namespace, that namespace MUST be defined and used in path construction for lxml to parse the XML file.  See <https://lxml.de/xpathxslt.html> for more details."""
        
        self.ns = {"atom":"http://www.w3.org/2005/Atom",
                   "cap":"urn:oasis:names:tc:emergency:cap:1.1",
                   "ha":"http://www.alerting.net/namespace/index_1.0"
                  }

## XMLHandler Class

This class contains methods used to process the NWS ATOM alert feed.  The Python lxml library is used to process the XML via XPath conventions for annotating tree structure.

Initialization of an instance of XMLHandler includes a namespace map.  With lxml it is *imperative* that all namespaces in the XML feed be defined in the **self.ns{}** dictionary, to include definition of the default namespace.  While this adds some complexity to the XPath definitions during processing, it ultimately provides an unambiguous tree processing convention which aids in rapid, clean XML handling.

The methods defined in this class handle data per the Common Alerting Protocol v1.1 standard.  For details on specific method data, see the <a href="https://www.oasis-open.org/committees/download.php/14759/emergency-CAPv1.1.pdf">OASIS CAP v1.1</a> standard documentation.

For methods where *enum* is used as an input variable, *enum* is simply a placeholder variable which was designed to be used with an iterator.  NWS alert feeds very frequently contain more than one alert event.  The *enum* construct allows simple iteration through a feed in the **main()** method of the program.

In [51]:
    def getEntryId(self, enum, default='N/A'):
        """Get feed entry ID number."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/atom:id/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces=self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryId Exception: {ex}')

### Entry ID

Each entry in the NWS ATOM feed has a unique entry ID.  This method retrieves that ID.

In [52]:
    def getEntryPublishedTime(self, enum, default='N/A'):
        """Get feed entry time published."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/atom:published/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces=self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryPublishedTime Exception: {ex}')

### Entry Published Time

Each entry in the NWS ATOM feed has a published time.  NOTE: the *published* time is differentiated from the entry *effective* time, which is the time the alert goes active.  This method retrieves the entry *published* time.

In [53]:
    def getEntryUpdatedTime(self, enum, default='N/A'):
        """Get feed entry time updated."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/atom:updated/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryUpdatedTime Exception: {ex}')

### Entry Updated Time

Entries which have already been published may need to be updated.  If the original entry ID remains the same, but the entry is updated, this DTG indicates the date/time of the most recent update.  This method retrieves the update time.

In [54]:
    def getEntryEventType(self, enum, default='N/A'):
        """Get feed entry event type."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:event/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventType Exception: {ex}')

### Entry Event Type

The NWS currently implements the Common Alerting Protocol v1.1 standard to format ATOM messages; however, several fields used by the NWS have additional specifications above those listed in the CAP standard.  The NWS utilizes a fixed list of available event types to identify event instances.  The <a href="https://alerts.weather.gov/cap/product_list.txt">most current list</a> is available at the NWS alert website.

In [55]:
    def getEntryEventEffective(self, enum, default='N/A'):
        """Get feed entry event effective DTG."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:effective/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventEffective Exception: {ex}')

### Entry Event Effective

The effective time is the time the alert goes active.  This method retrieves the effective time.

In [56]:
    def getEntryEventExpires(self, enum, default='N/A'):
        """Get feed entry event expires DTG."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:expires/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventExpires Exception: {ex}')

### Entry Event Expires

The expiration time is the time the alert period ends.  This method retrieves the expiration time.

In [57]:
    def getEntryEventStatus(self, enum, default='N/A'):
        """Get feed entry event status."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:status/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventStatus Exception: {ex}')

### Entry Event Status

The entry event status indicates the status of the entry event.  This method retrieves the event status.

In [58]:
    def getEntryEventMessageType(self, enum, default='N/A'):
        """Get feed entry event message type."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:msgType/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventMessageType Exception: {ex}')

### Entry Event Message Type

The entry event message can have several different administrative message types (Alert, Cancel, Update, etc.).  This method retrieves the event message type.

In [59]:
    def getEntryEventCategory(self, enum, default='N/A'):
        """Get feed entry event category."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:category/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventCategory Exception: {ex}')

### Entry Event Category

The entry event can be one of several different categories (Meteorological, Safety, Rescue, etc.).  The NWS feed consists primarily of MET category alerts; however, all types specified by the CAP v1.1 standard are recognized and displayed by APRS-NWS in the event NWS channels are used to enable widespread dissemination of non-metro alert data.  This method retrieves the event category.

In [60]:
    def getEntryEventUrgency(self, enum, default='N/A'):
        """Get feed entry event urgency."""
        self.enum = enum
        
        try:           
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:urgency/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventUrgency Exception: {ex}')

### Entry Event Urgency

The entry event has a specified forecast urgency.  This method retrieves the event urgency.

In [61]:
    def getEntryEventSeverity(self, enum, default='N/A'):
        """Get feed entry event severity."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:severity/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventSeverity Exception: {ex}')

### Entry Event Severity

The entry event has a specified forecast severity.  This method retrieves the severity.

In [62]:
    def getEntryEventCertainty(self, enum, default='N/A'):
        """Get feed entry event severity."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:certainty/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventCertainty Exception: {ex}')

### Entry Event Certainty

The entry event has a specified forecast certainty.  This method retrieves the certainty.

In [63]:
    def getEntryEventPolygon(self, enum, default='N/A'):
        """Get feed entry event polygon.  Returns a string of lat/lon in the format (-)DD.DD (-)DDD.DD,(-)DD.DD (-)DDD.DD (i.e. lat/lon are comma-separated...lat/lon pairs are space-separated.)"""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:polygon/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventPolygon Exception: {ex}')

### Entry Event Polygon

Some NWS alerts come with a polygon data set of lat/lon coordinates used to define a broader warning area.  This method retrieves the polygon coordinate set.  Future functionality using this data will include creating an APRS area object from the polygon data set.

In [64]:
    def getEntryEventAreaDescription(self, enum, default='N/A'):
        """Get feed entry event area description.  Returns a string of semi-colon separated standardized area descriptions."""
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:areaDesc/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventAreaDescription Exception: {ex}')

### Entry Event Area Description

The NWS utilizes standard codes to define alert area zones/counties.  Each zone/county code is accompanied by a standard textual definition of the alert zone/county.  This method retrieves the textual alert area description.  NOTE: Many alerts have more than one area description, as the alerts are often issued for more than one zone/county at a time.  This method returns all areas as a space-delineated text string.

In [65]:
    def getEntryEventGeocodeValue(self, enum, default='N/A'):
        """Get CAP geocode area values for individual warning/alerts areas.  Uses NWS-standardized zone/county descriptors.  """
        self.enum = enum
        
        try:
            self.path = 'atom:entry[' + str(self.enum) + ']/cap:geocode/atom:value[2]/text()'
            self.i = getAlerts(LOCALE).xpath(self.path, namespaces = self.ns)
            return self.i if self.enum else default
        
        except Exception as ex:
            print(f'getEntryEventGeocodeValue Exception: {ex}')

### Entry Event Geocode Value

The NWS utilizes standard geocodes to define alert area zones/counties.  This method retrieves the geocodes.  NOTE:  Many alerts have more than one geocode, as the alerts are often issued for more than one zone/county at a time.  This method returns all geocodes as a space delineated text string.  The **main()** method of the program splits the group of geocodes and issues an APRS message alert for each geocode individually to allow end users to implement message filter functionality of most APRS clients to receive alerts only for zones/counties which concern them.

In [66]:
class MsgHandler:
    """Methods for creating properly formatted APRS messages.  See <http://www.aprs.org/doc/APRS101.PDF> for baseline message specifications.  Follow-on specifications changes are documented at <http://www.aprs.org/aprs11.html> and proposed improvements at <http://www.aprs.org/aprs12.html>."""
    
    def __init__(self):
        """Initializes message handler to create APRS-formatted messages.  Default text encoding is UTF-8."""
        self.encoding = 'utf-8'

## MsgHandler Class

This class contains methods to create properly formatted APRS messages in accordance with published APRS standards.

In [67]:
    def makeToCall(self, geocode):
        """Create a 9-byte to-call for weather alert messages using the format XXXXXX___, where XXXXXX is a standardized NWS six-byte alert zone or county identifier, followed by three spaces.  Per the APRS specification, the to-call is a fixed 9-byte address."""
        self.geocode = geocode
        
        try:
            if len(self.geocode) <= 6:
                self.tocall = str(self.geocode + '   ')
            else:
                self.tocall = self.geocode.split(' ')
            return self.tocall
        
        except Exception as ex:
            print(f'makeToCall Exception: {ex}')

### Make ToCall

APRS messages are addressed using a 9-bit "to-call" (not to be confused with the AX.25 UI packet TOCALL) which can be thought of as the address to which the message is sent.  With station-to-station messaging, the to-call is normally the callsign+SSID of the station to whom the APRS message is sent; however, the APRS standard also makes provision for "group messaging" where a message is addressed to a group address and client hardware/software can be configured with filters to only display group messages of interest.  

Historically, many clients have displayed *NWS*-addressed messages by default, as the NWS message is a defined group message in the APRS v1.01 specification.  Unfortunately, this often causes distant stations to receive NWS message alerts for weather events that are not in their local area/area of interest.

APRS-NWS implements a new convention where the *NWS* message from APRS v1.01 specification is modified using the NWS-assigned six-byte county/zone geocode as the "to-call" for the message.  Fucntionally, this creates a group message to that zone/county.  Users can then configure their APRS client to display one or more zones/counties of interest.  Many clients also allows the used of a wildcard character, so users who wish to view alerts can simply adjust their message filter in the following manner:

**Zone-level Filter Setup**
The zone of interest is NMZ414.  The end-user does not want to receive any other NWS alerts.  Client group message filter is set to:  **NMZ414**

**Multi-Zone Filter Setup**
The zones of interest are NMZ414 and NMC035.  The end-user does not want to receive any other NWS alerts.  Client group message filter is set to:  **NMZ414**,**NMC035**

**State-Level Filter Setup**
The end-user is interested in receiving all alerts for the state of New Mexico.  Client group message filter is set to:  **NM*** (where * is the client hardware/software wildcard character)

**Multi-State Filter Setup**
The end-user is interested in receiving all alerts for the states of New Mexico and Texas.  Client group message filter is set to:  **NM***,**TX*** (where * is the client hardware/software wildcard character)

In [68]:
    def makeWxMsgPacket(self, evttype, atype, severity, certainty, urgency, category, eff_start, eff_end):
        """Create APRS weather alert message."""
        self.evttype = evttype
        self.atype = atype
        self.severity = severity
        self.certainty = certainty
        self.urgency = urgency
        self.category = category
        self.eff_start = eff_start
        self.eff_end = eff_end
        
        try:
            self.msg_pkt = (f'*{self.evttype}* {self.atype} {self.severity}-{self.certainty}-{self.urgency} {self.category} {self.eff_start}-{self.eff_end}')
            if len(self.msg_pkt) <= 78:      # Maximum allowable length of the APRS NWS message minus message ID length is 78 characters.
                return self.msg_pkt
            else:
                print('The message packet is too long...truncating.')
                return self.msg_pkt[:79]
            
        except Exception as ex:
            print(f'makeWxMsgPacket Exception: {ex}')

### Make Weather Message Packet

Method to assemble parsed/formatted data into a valid APRS message, using the APRS-NWS specification for a weather alert packet.  Contains a length validation check to ensure that total packet length complies with the APRS v1.01 standard for an NWS alert packet.  Excessively long packets are still formed, but truncated with alerting.

In [69]:
    def appendMsgId(self, msg):
        """Append a valid APRS message ID to a constructed message using the Python hash() function.  NOTE: hash() can only be used on immutable Python objects.  Returns the entire message w/ ID."""
        self.msg = msg
        
        try:
            self.h = str(hash(self.msg))
            self.m_id = self.h[-5:]
            self.t_msg = str(self.msg + '{' + self.m_id)
            return self.t_msg
        
        except Exception as ex:
            print(f'appendMsgId Exception: {ex}')

### Append Message ID

APRS messages can be sent with a 5-byte message ID appended to the end of the message.  This ID is commonly used to mitigate unnecessary duplicate packets in the system at various points.  The Python **hash()** function is used here to generate an ID; however, there are two major things to note about the implementation of the hash function:

1. The **hash()** function can only be used on immutable objects (like tuples).
2. The hash for a message will remain the same as long as it is computed under the same running instance.  If the program stops and is restarted, the hash for the same ASCII string will be different.

Other methods such as CRC checks can also be used to generate a suitable message ID if the above constraints do not meet end-user needs.

In [70]:
def main():
    """Main program."""
    x = XMLHandler()
    m = MsgHandler()
    
    try:
        if os.path.exists('/tmp/wxalerts.txt'):
            os.remove('/tmp/wxalerts.txt')

SyntaxError: unexpected EOF while parsing (<ipython-input-70-1edb27e1ded1>, line 8)

## Main Program

The main program creates a .txt file at */tmp/wxalerts.txt* if one doesn't already exist, or deletes and replaces the file if it does exist.  This step is critical to the implementation of APRS-NWS with the APRX software suite.  The separate companion script *aprxfeeder.py* reads the file */tmp/wxalerts.txt* and writes the top line of the file to *stdout*, then deletes the line.  APRX takes the output to *stdout* and transmits it as a raw APRS packet.  Since APRX is unable to process a multi-line file, this process is repeated at an interval defined in the */etc/aprx.conf* file (5 minutes is the default interval).

Future functionality will eliminate the implementation of the .txt file and instead pass the packet over a TCP port directly to APRX for transmission.

In [ ]:
        e = 1
        while e <= entries:
            zones = x.getEntryEventGeocodeValue(e)
            evttype = x.getEntryEventType(e)
            atype = x.getEntryEventMessageType(e)
            severity = x.getEntryEventSeverity(e)
            certainty = x.getEntryEventCertainty(e)
            urgency = x.getEntryEventUrgency(e)
            category = x.getEntryEventCategory(e)
            evt_start = x.getEntryEventEffective(e)
            evt_end = x.getEntryEventExpires(e)
                        
            #Convert raw data to APRS-formatted data
            f_evttype = cap.makeEventType(evttype)
            f_alt_type = cap.makeAlertType(atype)
            f_severity = cap.makeSeverity(severity)
            f_certainty = cap.makeCertainty(certainty)
            f_urgency = cap.makeUrgency(urgency)
            f_category = cap.makeCategory(category)
            f_evt_start = cap.makeEffectiveStart(evt_start[0])
            f_evt_end = cap.makeEffectiveEnd(evt_end[0])
            
            wxmsg = m.makeWxMsgPacket(f_evttype, f_alt_type, f_severity, f_certainty, f_urgency, f_category, f_evt_start, f_evt_end)

### Creating a Weather Alert Packet

The process for assembling the packet is fairly straightforward.  The XML feed is parsed, the data is mangled into the APRS-NWS format alert message via the CAP library, and finally the packet is assembled using the appropriate MsgHandler method.

In [ ]:
            zonelist = zones[0].split(' ')
            for z in zonelist:
                tocall = m.makeToCall(z)
                zonetext = nm.makeNMZoneText(z)    #TODO:  this will have to change when getAlerts is modified for multiple locales
                fullmsg = (f':{tocall}:{zonetext} ' + wxmsg)
                with open('/tmp/wxalerts.txt', 'a') as f:
                    f.writelines(m.appendMsgId(fullmsg) + '\n')
                sys.stdout.write(m.appendMsgId(fullmsg))
            e += 1
            
    except Exception as ex:
        print(f'Main Exception: {ex}')
        pass

### Multi-Zone Handling

The NWS frequently issues an alert for more than one zone/county.  This part of the **main()** function splits the multi-entry zone data obtained earlier in the program and creates an individual alert message for each zone/county.  The individual zone label is also used as the 'tocall' for the APRS message, which is what allows end users to filter out unwanted data using client group message filtering features.

Sysops of APRS-NWS should monitor packet transmission rates, as areas with a large number of alerts may oversaturate a 1200baud APRS channel.  Sysops in these areas should give consideration to using an alternate channel, or a 9600baud APRS channel if one is available in that area.

In [ ]:
    time.sleep(1800)
            
if __name__ == '__main__':
    while True:
        main()
        time.sleep(14400)


### Sleep Function

The APRS-NWS main method is set to run every 30 minutes (1800 seconds) by default.  This rate should be more than adequate for most alert areas, but may be adjusted more/less frequently based on localized conditions.  Note:  the rate set here is the rate at which the */tmp/wxalerts.txt* file is re-created.  The rate at which APRX transmits individual packets is set at */etc/aprx.conf*.  Keep in mind that the *aprxfeeder.py* script deletes lines in the *wxalerts.txt* file, so a slow update rate here and a rapid transmit rate in APRX will have no effect, as APRX will have no lines to transmit once *aprxfeeder.py* removes those lines from the alerts text file.  Sysops should find a balance between both rates suitable for local weather and available bandwidth conditions.